# Basic PyTorch Text Classification

## Overview
This tutorial is extracted from https://github.com/bentrevett/pytorch-sentiment-analysis. Before proceeding, make sure to install the required library. 


# 准备数据

- 原始例子用的是spacy，但是这个库在谷歌云上会有一个Bug，因此改成了nltk；
- 英文中的分词主要是对缩写的处理

In [ ]:
import nltk
nltk.download('punkt')

In [ ]:
from nltk.tokenize import word_tokenize
tokenizer = word_tokenize

In [11]:
import torch
from torchtext import data
from torchtext import datasets

SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize = tokenizer, include_lengths = True)
LABEL = data.LabelField(dtype = torch.float)

In [ ]:
from torchtext import datasets

train_data, test_data = datasets.IMDB.splits(TEXT, LABEL, root='F:\\')

In [12]:
import random

train_data, valid_data = train_data.split(random_state = random.seed(SEED))

In [13]:
train_data.shape

<generator object Dataset.__getattr__ at 0x000001C206821F90>

# 预处理

In [14]:
MAX_VOCAB_SIZE = 25000

TEXT.build_vocab(train_data, 
                 max_size = MAX_VOCAB_SIZE, 
                 vectors = "glove.6B.300d", # 根据6个词来计算概率（如果词评率排在25000之外的话，则不参与计算）；300维的向量；
                 unk_init = torch.Tensor.normal_) # 对于没有出现的词，随机初始化词向量

LABEL.build_vocab(train_data)

.vector_cache\glove.6B.zip: 862MB [17:10, 837kB/s]                                                                     
100%|███████████████████████████████████████████████████████████████████████▉| 399999/400000 [00:47<00:00, 8378.83it/s]


In [ ]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# BucketIterator就是上一节中的Dataloader
train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    sort_within_batch = True,
    device = device)

# 定义网络

In [ ]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, 
                 bidirectional, dropout, pad_idx):
        
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        
        self.rnn = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           bidirectional=bidirectional, 
                           dropout=dropout)
        
        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text, text_lengths):
        
        embedded = self.embedding(text)
        
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths)
        
        packed_output, (hidden, cell) = self.rnn(packed_embedded)
        
        output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output) 
        
        hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)) # 双向LSTM
                
        return self.fc(hidden)

![Packed Sequence](pic\rnn_packed_seq.jpg)

In [ ]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 300
HIDDEN_DIM = 256
OUTPUT_DIM = 1
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.2
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = RNN(INPUT_DIM, 
            EMBEDDING_DIM, 
            HIDDEN_DIM, 
            OUTPUT_DIM, 
            N_LAYERS, 
            BIDIRECTIONAL, 
            DROPOUT, 
            PAD_IDX)

In [ ]:
pretrained_embeddings = TEXT.vocab.vectors
model.embedding.weight.data.copy_(pretrained_embeddings)


In [ ]:
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

In [ ]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters()) #把模型中所有的权重都给了优化器
criterion = nn.BCEWithLogitsLoss() # 先把模型输出转换为0-1之间的概率形式（logits变化/softmax）,然后计算binary cross entropy(BCE),也叫log loss

model = model.to(device)
criterion = criterion.to(device)

In [ ]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [ ]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        text, text_lengths = batch.text
        
        predictions = model(text, text_lengths).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            text, text_lengths = batch.text
            
            predictions = model(text, text_lengths).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
N_EPOCHS = 5

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

In [ ]:
model.load_state_dict(torch.load('model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

# 模型优化
- 分词器：对于英文来说，分词的影响没有特别大
- 数据清理
    - 不是数据越干净，效果会越好
    - 相同的数据清洗方式在不同的模型上，效果不同
    - 先处理明显的错误（例如去除过多空格）
    - 先在最好的模型上尝试数据清理
- train和validation数据集该怎么挑选
    - test data要有代表性
    - train一般比validation要大
    - 数据量比较大的时候（十几万），需要做五折验证
    - 竞赛时，建议看测试集的结果，多提交代码。（很多比赛数据集质量比较差）
- MAX_VOCAB_SIZE的选择
    - 会影响结果的比较重要的参数；
    - 一般会尽量都使用所有的embedding
    - 如果单纯训练embedding的话，出现频率特别小的词，可以不去训练，或者所有低频率词都训练为一个token；
    - 常用trick: 在目标训练集或者测试集上，稍微再训练一下word embedding，（learning rate选择小一点）
    - 词向量怎么选：需要测试，通常会选择较大的词向量因为词向量的表现力一般是比较差；为了让模型更丰富，可以多个词向量拼接来跑，但如果都是词向量的话，提升往往不会很大，但是如果有BERT或者ELMO的话，与词向量结合会有比较大的效果提升；
    - 对于CV来说，上层模型影响比较大；但是对于NLP来说，上层模型的影响有限；
- 为什么不使用torchtext？
    - torchtext没有办法处理BERT和词向量混用的问题（尤其是中文）：BERT是字向量。
    
- batch size
    - 一般不会用太大的batch size， 32， 64， 128
    - 如果用特别大的值，耗费资源，收敛慢；
    - 开始时用小的batch size,快收敛时，调低learning rate,调高batch size;
    - 机器翻译的batch size一般比较大；
    - 如果模型比较大，可能会有显存问题，解决方法：半精度计算（例如梯度存储32位浮点数-->16）；分布式计算（多卡，哪些数据要在每一个卡存一遍；卡与卡之间的交流；微软开发的DeepSpeed库帮忙解决了部分问题）；梯度累积（速度不会加快，Pytorch做梯度累积时会有累积误差，导致精度下降，建议累积不超过2步）
    
- LSTM层数
    - 叠加LSTM不是一个很好的方式：速度会受比较大的影响；精度提升不大，LSTM擅长处理文本梯度传播，而不是多层LSTM，但是也不一定；
    
- Dropout
    - 一般比较小
- Hidden Dim
    - 比较重要的参数
- Seq Length
    - 需要调试，比较重要
- Learning rate
    - warmup：开始使用小学习率，后面增大，再后面降下来
    - 如果开始使用小学习率，会改变参数分布比较多；
- 初始化
    - LSTM初始化比较容易
    - 不同的初始化的结果：随机初始化比全0好；正态初始化比均匀分布初始化好，但是方差怎么选是个问题，kaiming初始化效果比较好；
- 优化器
    - learning rate一定要调试，一开始会比较大
    - 不推荐Embedding和上层模型一起训练，一开始固定Embedding来训练；
    - 快收敛时，要么单独训练Embedding或者使用小学习率，一起训练Embedding和模型；
- Loss
    - 多个Loss加起来可能有的好处：初始阶段加多个Loss，收敛可能会加快；快收敛时加多个Loss，可能可以收敛到一个更好的结果；
    - 多个Loss时，每个loss的权重很重要；
- Train
    - 不推荐在每个Epoch结尾才做evaluate，一般在跑50或100个batch后做evaluate；
    - 模型存储：不能只存最好的模型；Babysitting Training Process (不断地改变优化器状态、网络的状态，人工改变准确性）
    - 除了模型状态，还应该保存优化器状态，Scheduler的状态（手动改变learning rate）
